In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%%capture
import pandas as pd
import networkx as nx
pd.set_option('display.max_columns', None) # so it prints all the dataframe columns ...

In [3]:
from pathlib import Path
import os
import glob
csv_df = pd.DataFrame(columns=[])
# for file in glob.glob("/content/drive/MyDrive/changed_CD012768*.csv"): # tested the whole thing only on "changed_CD012768.csv" and "changed_CD012768 (copy).csv" ...
for file in glob.glob("/content/drive/MyDrive/Feature_extraction_results/*.csv"):
  filename = os.path.basename(file)
  temp_df = pd.read_csv(file, index_col=0)
  temp_df[ "Filename" ]=filename
  if ( "'int'" in str(type(csv_df)) ):
    csv_df = temp_df.copy()
  else:
    csv_df = pd.concat([csv_df, temp_df], axis=0, ignore_index=True)
  print( "All " + str(temp_df.shape[0]) + " rows of \"" + file + "\" were added to \"csv_df\" .. " )
  # print( "\"csv_df\" has " + str(csv_df.shape[0]) + " rows ..." )
  # csv_df = csv_df.drop_duplicates()
  print( "\"csv_df\" has " + str(csv_df.shape[0]) + " rows after duplicate removal ..." )
  print( "===========" )

print( "Total count of of \"PMID\" values:\t" + str(csv_df.shape[0]) )
print( "Duplicate count of \"PMID\" column:\t" + str(csv_df.duplicated(subset="PMID").sum()) )

pmids = list(csv_df["PMID"])
pmids = list(dict.fromkeys(pmids)) # remove duplicate PMID values
print( "Number of distinct \"PMID\" values:\t" + str(len(pmids)) )

All 119 rows of "/content/drive/MyDrive/Feature_extraction_results/finished_new_prompt_CD012768_soft.csv" were added to "csv_df" .. 
"csv_df" has 119 rows after duplicate removal ...
All 119 rows of "/content/drive/MyDrive/Feature_extraction_results/finished_new_prompt_CD012768_hard.csv" were added to "csv_df" .. 
"csv_df" has 238 rows after duplicate removal ...
All 119 rows of "/content/drive/MyDrive/Feature_extraction_results/ada_embed_CD012768.csv" were added to "csv_df" .. 
"csv_df" has 357 rows after duplicate removal ...
All 119 rows of "/content/drive/MyDrive/Feature_extraction_results/finished_new_prompt_CD012768.csv" were added to "csv_df" .. 
"csv_df" has 476 rows after duplicate removal ...
All 1799 rows of "/content/drive/MyDrive/Feature_extraction_results/ada_embed_CD008874.csv" were added to "csv_df" .. 
"csv_df" has 2275 rows after duplicate removal ...
All 1799 rows of "/content/drive/MyDrive/Feature_extraction_results/finished_new_prompt_CD008874.csv" were added to "c

In [4]:
csv_df = csv_df.astype(str)

In [ ]:
no_ref_positive = csv_df[ (csv_df["references"] == "[]") & (csv_df["Status"] == "1") ]
no_ref_negative = csv_df[ (csv_df["references"] == "[]") & (csv_df["Status"] == "0") ]
print(f'There are {len(no_ref_positive)} postive articles with no references and {len(no_ref_negative)} negative articles with no references.')
# no_ref_positive["Filename"].value_counts() # prints how many positive no ref per file
# no_ref_negative["Filename"].value_counts() # prints how many negative no ref per file

There are 254 postive articles with no references and 22806 negative articles with no references.


In [ ]:
csv_df.shape

(105521, 56)

In [5]:
csv_df_copy = csv_df.copy(deep=True)

In [15]:
csv_df = csv_df_copy.copy(deep=True)

In [6]:
csv_df["references"].describe()

,references
count,105521
unique,20943
top,[]
freq,23091


In [7]:
csv_df.shape

(105521, 56)

In [16]:
def get_empty(row):
  if ( row["references"] == "[]" ):
    return row["PMID"]
  if ( str(row["references"]) == "nan" ):
    return row["PMID"]
  return ""

empty_row_iter = csv_df.apply( get_empty, axis=1 )
empty_row_iter = [x for x in empty_row_iter if x != ""]
csv_df = csv_df[~csv_df["PMID"].isin(empty_row_iter) ] # here ~ is equivalent to NOT (negates the statement)

In [9]:
csv_df.shape

(82426, 56)

In [10]:
len(empty_row_iter)

23095

In [27]:
# you might have the same reference formulated differently amongst more rows ...
# since there are references that have no unique identifier (doi/pmid) maybe it would be best
# if i did an NLP comparison between em to see if they are the same ...
# this would then be a citation network that is based on citation SIMMILARITY not identicality which might NOT have been done yet ...
# !pip install fix-busted-json
# from fix_busted_json import repair_json

import json
!pip install demjson3
from demjson3 import decode
from IPython.display import clear_output

missed = []
def get_json_citation(row):
  global missed
  succeeded = True

  try:
    references_json = decode(row["references"])
    citations=[]
    for ref in references_json:
      # clear_output(wait=True)
      # print( ref )
      citations.append( ref["citation"] )
    return json.dumps(citations)
  except:
    missed.append( row["references"] )

# if ( not succeeded ):
#   try:
#     missed.append( row["references"] )
#     references_json = repair_json(row["references"])
#     citations=[]
#     for ref in references_json:
#       clear_output(wait=True)
#       print( ref )
#       citations.append( ref["citation"] )
#     return json.dumps(citations)
#   except:
#     missed.append( row["references"] )
#     clear_output(wait=True)
#     print(len(missed))
#     return "[]"

# print(missed)

filenames = csv_df["Filename"].unique().tolist()
cit_dfs = []
all_citations = []
for filename in filenames:
  temp_df = csv_df[ csv_df["Filename"] == filename ]
  print( f'temp: {temp_df.shape}' )
  temp_df["reference_array"] = temp_df.apply( get_json_citation, axis=1 )
  all_citations.extend( temp_df["reference_array"].tolist() )
  file_df = temp_df[["PMID", "citation", "reference_array"]]
  cit_dfs.append(file_df)

# print(cit_dfs)

temp: (106, 56)


<ipython-input-27-0b7bcfdfb7b7>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["reference_array"] = temp_df.apply( get_json_citation, axis=1 )


temp: (106, 56)


<ipython-input-27-0b7bcfdfb7b7>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["reference_array"] = temp_df.apply( get_json_citation, axis=1 )


temp: (106, 56)


...

<ipython-input-27-0b7bcfdfb7b7>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["reference_array"] = temp_df.apply( get_json_citation, axis=1 )


temp: (987, 56)


<ipython-input-27-0b7bcfdfb7b7>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["reference_array"] = temp_df.apply( get_json_citation, axis=1 )


temp: (987, 56)


<ipython-input-27-0b7bcfdfb7b7>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df["reference_array"] = temp_df.apply( get_json_citation, axis=1 )


In [28]:
cit_dfs0 = cit_dfs[0].copy(deep=True)
cit_dfs0

,PMID,citation,reference_array
0,22815718,"Peter JG, et al. The diagnostic accuracy of ur...","[""The American review of respiratory disease B..."
1,27286562,"Held M, et al. Diagnostic Accuracy of the Xper...","[""Indian J Orthop Anley 46 186 2012 10.4103/00..."
2,22381459,"Chang K, et al. Rapid and effective diagnosis ...","[""Expert Rev Anti Infect Ther Steingart 5 327 ..."
4,24167451,"Patel VB, et al. Diagnostic accuracy of quanti...","[""Clin Microbiol Rev CK Kwan 24 351 2011 10.11..."
5,27366339,"Hodkinson B, et al. HIV Infection and Osteoart...","[""10.1016/j.tube.2013.04.003"", ""10.1016/j.clin..."
...,...,...,...
112,25385864,"Rajasingham R, et al. Epidemiology of meningit...","[""WHO Health Topics: Meningitis. 2013. http:/..."
113,22734954,Lawn SD and Zumla AI. Diagnosis of extrapulmon...,"[""Future Microbiol. Lawn 1067 6 2011 10.2217/f..."
114,25767518,"Du J, et al. Rapid diagnosis of pleural tuberc...","[""Geneva, Switzerland: World Health Organizati..."
115,26654321,"Marouane C, et al. Evaluation of molecular det...","[""J Clin Microbiol Causse 49 3065 2011 10.1128..."


In [66]:
cit_dfs[0] = cit_dfs0.copy(deep=True)

In [64]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt') # if necessary...
nltk.download('punkt_tab')

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


cosine_sim('PLoS One Claassens 8 e76272 2013 10.1371/journal.pone.0076272 Tuberculosis in healthcare workers and infection control measures at primary healthcare facilities in South Africa', 'van Soolingen D, Hermans PW, de Haas PE, Soll DR, van Embden JD. Occurrence and stability of insertion sequences in Mycobacterium tuberculosis complex strains: evaluation of an insertion sequence-dependent DNA polymorphism as a tool in the epidemiology of tuberculosis. Journal of clinical microbiology. 1991;29(11):2578–86.')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli

0.039212891527794676

In [ ]:
all_refs = []
all_refs_unique = []

def get_refs(row):
  global all_refs
  ref_json = decode(row["reference_array"])
  for ref in ref_json:
    if ( ref not in all_refs ):
      all_refs.append(ref)

def get_cosine_simmilarity(ref):
  count = 1
  for u in all_refs_unique:
    if ( cosine_sim( ref, u ) >= 0.05 ): # adjust the treshold here ...
      count = count + 1
  return count

cit_dfs[0].apply( get_refs, axis=1 )
all_refs_unique = list(set(all_refs))

def get_ref_freq(row):
  global all_refs
  global all_refs_unique
  ref_json = decode(row["reference_array"])
  freq = []
  for ref in ref_json:
    # freq.append( '{ citation:"' + ref + '", freq:' + str(all_refs.count(ref)) + ", id:" + str(all_refs_unique.index(ref)) + "}")
    temp_json = {}
    temp_json["citation"] = ref
    # temp_json["freq"] = str(all_refs.count(ref))
    temp_json["freq"] = str(get_cosine_simmilarity(ref))
    temp_json["id"] = str(all_refs_unique.index(ref))
    freq.append( temp_json )
  return freq

# for cit_df in cit_dfs:
all_refs = []
all_refs_unique = []
cit_dfs[0].apply( get_refs, axis=1 )
all_refs_unique = list(set(all_refs))
cit_dfs[0]["reference_array"] = cit_dfs[0].apply( get_ref_freq, axis=1 )

cit_dfs[0]

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [58]:
cit_dfs[0] = cit_dfs[0].astype(str)

def print_refs(row):
  global all_refs
  ref_json = decode(row["reference_array"])
  for ref in ref_json:
    # print(ref)
    # ref = decode(ref)
    print(ref["id"])
    print(ref["citation"])
    print(ref["freq"])
    if ( str(ref["freq"]) != "1" ): # they are ALL 1 ... so, I would want to make a NLP-type comparison of them, to have a frequency that is NOT 1 ...
      print( "NOT 1" )
      print(1 + "1")
    print("\n\n")

cit_dfs[0].apply( print_refs, axis=1 )

Streaming output truncated to the last 5000 lines.


444
Becton Dickinson Diagnostic Systems. BD MGITTM TBc ID identification test package insert, BD document L8085917(01). Sparks, MD: BD Diagnostic Systems; 2009.
1



1429
van Soolingen D, Hermans PW, de Haas PE, Soll DR, van Embden JD. Occurrence and stability of insertion sequences in Mycobacterium tuberculosis complex strains: evaluation of an insertion sequence-dependent DNA polymorphism as a tool in the epidemiology of tuberculosis. Journal of clinical microbiology. 1991;29(11):2578–86.
1



746
Hain Lifescience. Protocole d'utilisation du test GenoType® MTBDRplus version 2.0. 2012  http://www.moleculartb.org/gb/pdf/protocols/Protocole_GenoType_MTBDRplus_V2.pdf. Accessed on June 15, 2016.
1



389
Ozkutuk N, Surucuoglu S. Evaluation of the Xpert MTB/RIF assay for the diagnosis of pulmonary and extrapulmonary tuberculosis in an intermediate-prevalence setting. Mikrobiyoloji bulteni. 2014;48(2):223–32.
1



944
Zmak L, Jankovic M, 

,0
0,None
1,None
2,None
4,None
5,None
...,...
112,None
113,None
114,None
115,None


In [ ]:
import json
data = {"pmid": [], "citation": []}
citation_df = pd.DataFrame(data)

def parse_citations(row):
  references_json = json.loads(row["references"])
  for ref in references_json:
    if ( "pubmed" in ref["article_ids"] ):
      citation_df.loc[len(citation_df)] = [row["pmid"], "PMID-"+str(ref["article_ids"]["pubmed"])]
      continue
    if ( "pmc" in ref["article_ids"] ):
      citation_df.loc[len(citation_df)] = [row["pmid"], "PMC-"+str(ref["article_ids"]["pmc"])]
      continue

    citation_df.loc[len(citation_df)] = [row["pmid"], "TITLE-"+str(ref["citation"])]
    continue

citations.apply( parse_citations, axis=1 )
citation_df = citation_df.astype(str)
citation_df

In [ ]:
citation_df.describe()

In [ ]:
# Citation network with ALL citations ...

citation_network = nx.from_pandas_edgelist(citation_df, "pmid", "citation")
# nx.draw_spring(citation_network) # draw_planar / draw / draw_spring - those are the best visualisation functions ...
print( "Number of connected components: " + str( nx.number_connected_components(citation_network) ) )

In [ ]:
# CODE FOR MAKING A CITATION NETWORK FOR EACH ARTICLE INDIVIDUALLY

# initial_article_pmids = list(citation_df['pmid'].unique())

# # article_dfs=[]
# for initial_article_pmid in initial_article_pmids:
#   article_df = citation_df[citation_df["pmid"]==initial_article_pmid]

#   citation_network = nx.from_pandas_edgelist(article_df, "pmid", "citation")
#   nx.draw_spring(citation_network) # draw_planar / draw / draw_spring - those are the best visualisation functions ...
#   print( "Number of connected components: " + str( nx.number_connected_components(citation_network) ) )

#   # article_dfs.append( article_df )

# article_df